In [1]:
from mismatchtree import mismatchTree, isInMismatchTree, inMismatchTree

import json

In [2]:
s = 'TAGG'

# Build mismatch tree

In [3]:
mismatch_tree_example = mismatchTree(s, 1)

In [4]:
print(json.dumps(mismatch_tree_example, indent=4, sort_keys=True))

{
    "A": {
        "A": {
            "G": [
                "G"
            ]
        }
    },
    "C": {
        "A": {
            "G": [
                "G"
            ]
        }
    },
    "G": {
        "A": {
            "G": [
                "G"
            ]
        }
    },
    "T": {
        "A": {
            "A": [
                "G"
            ],
            "C": [
                "G"
            ],
            "G": [
                "T",
                "A",
                "C",
                "G"
            ],
            "T": [
                "G"
            ]
        },
        "C": {
            "G": [
                "G"
            ]
        },
        "G": {
            "G": [
                "G"
            ]
        },
        "T": {
            "G": [
                "G"
            ]
        }
    }
}


In [5]:
'TACG'

'TACG'

In [6]:
isInMismatchTree(mismatchTree('TACG', 1), 'TACG')

True

In [7]:
isInMismatchTree(mismatchTree('TACC', 1), 'TAGG')

False

In [8]:
inMismatchTree(mismatchTree('TACC', 1), ['TAGG', 'TGGG', 'TACG', 'TGCC'])

['TACG', 'TGCC']

In [9]:
mismatchTree('TACC', 1)

{'A': {'A': {'C': ['C']}},
 'C': {'A': {'C': ['C']}},
 'G': {'A': {'C': ['C']}},
 'T': {'A': {'A': ['C'], 'C': ['T', 'A', 'C', 'G'], 'G': ['C'], 'T': ['C']},
  'C': {'C': ['C']},
  'G': {'C': ['C']},
  'T': {'C': ['C']}}}

# Speed test

In [10]:
%load_ext line_profiler

In [11]:
mismatch_tree = mismatchTree('TACC', 1)

In [23]:
%lprun -f isInMismatchTree isInMismatchTree(mismatch_tree, 'TAGG')

# Debugging treetest

In [29]:
from kernel import build_spectrum_kernel_matrix
s1 = 'AAAA'
s2 = 'AATA'
kernel_parameters = {'k':3,'m':1}
K = build_spectrum_kernel_matrix([s1,s2],kernel_parameters)
K

Building kernel matrix: 100%|██████████| 2/2 [00:00<00:00, 1292.94it/s]

{(1, 'AAT'): ['AAT'], (0, 'AAA'): ['AAA'], (0, 'AAT'): ['AAA'], (1, 'ATA'): ['ATA'], (0, 'ATA'): ['AAA']}


array([[4., 4.],
       [4., 2.]])

In [4]:
from kernel import get_spectrum

In [23]:
spectrum_1 = get_spectrum(s1, k=3)
spectrum_1

['AAA', 'AAA']

In [24]:
spectrum_2 = get_spectrum(s2, k=3)
spectrum_2

['AAT', 'ATA']

In [25]:
from mismatchtree import mismatchTree, inMismatchTree
mt_1 = mismatchTree('AAA', 1)
mt_2 = mismatchTree('AAT', 1)
mt_3 = mismatchTree('ATA', 1)

In [27]:
inMismatchTree(mt_1, spectrum_1)

['AAA', 'AAA']

In [28]:
inMismatchTree(mt_2, spectrum_2)

['AAT']

# Mismatch String Kernel
(https://papers.nips.cc/paper/2179-mismatch-string-kernels-for-svm-protein-classification.pdf)

Alphabet $\mathcal{A} = \{T, C, A, G\}$

Mismatch feature vector for a k-mer $\alpha$ : 

\begin{equation}
\Phi_{(k,m)}(\alpha) = (\phi_\beta(\alpha))_{\beta \in \mathcal{A}^4}
\end{equation}

Feature vector for sequence $x$ of any length (in our case DNA sequence) : 
\begin{equation}
\Phi_{(k,m)}(x) = \sum_{\text{k-mers } \alpha \text{ in } x} \Phi_{(k,m)}(\alpha)
\end{equation}

### Exemple à la main:

$s_1 =$ AAAA

Spectrum($s_1$, 2) = [AAA, AAA]

$s_2 =$ AATA

Spectrum($s_2$, 2) = [AAT, ATA]

$\Phi_{(3, 1)}("AAA") = 1$ pour $\beta \in ["AAA", "AAT", "ATA", "TAA", ...]$ 


D'où 
\begin{equation}
\Phi_{(3, 1)}(s_1) = 2 \times \Phi_{(3, 1)}("AAA")
\end{equation}

Et 
\begin{equation}
\begin{split}
K(s_1, s_2)  &= \langle \Phi_{(3, 1)}(s_1), \Phi_{(3, 1)}(s_1) \rangle \\
&= 4 \times \langle \Phi_{(3, 1)}("AAA"), \Phi_{(3, 1)}("AAA") \rangle \\
&= 4 \frac{4!}{(4-m)!} \times \frac{k!}{(k-m)!} \\
&= 48
\end{split}
\end{equation}

Je me suis peut être trompé sur la dernière égalité en gros pour moi le nombre de coefficients non nuls c'est $\frac{4!}{(4-m)!} \times \frac{k!}{(k-m)!} $ car il faut choisir :
- quelles lettres remplacer : $\frac{k!}{(k-m)!}$ (arrangement m parmi k)
- par quelles lettres on les remplace : $\frac{4!}{(4-m)!}$ (arrangement m parmi 4 car 4 lettres dans notre alphabet)